In [ ]:
from pathlib import Path
import cv2
from video_player import (
    Video, 
    JupyterPlayer, 
    load_bounding_boxes_from_csv, 
    canny_edge_detector,
    DesktopPlayer,
    np_to_overlay_items,
    Point
)

In [ ]:
video_path = str(Path().resolve().parent.parent / "testing" / "resources" / "sample.mp4")
annotations_path = str(Path().resolve().parent.parent / "testing" / "resources" / "annotations.csv")
output_path = str(Path().resolve().parent.parent / "testing" / "output")

In [ ]:
def get_jupyter_player():
    """Returns a configured JupyterPlayer instance for interactive use."""
    video = Video(video_path)
    
    # Add named transformations
    video.add_transform(name="Grey scale", transform_func=lambda frame: cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    video.add_transform(name="Canny Edges", transform_func=lambda frame: cv2.Canny(frame, 100, 200))
    video.add_transform(name="Threshold", transform_func=lambda frame: cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)[1])
    video.add_transform(name="resize", transform_func=lambda frame: cv2.resize(frame, (960 // 2, 540 // 2)))

    video.add_online_overlay(
        name="Features", 
        overlay_func=lambda frame: np_to_overlay_items(
            cv2.goodFeaturesToTrack(frame, maxCorners=50, qualityLevel=0.01, minDistance=10, blockSize=3),
            overlay_item=Point,
        ),
    )

    # Load bounding boxes from the CSV file with a specific overlay name
    overlays = load_bounding_boxes_from_csv(annotations_path, overlay_name="CSV Boxes")
    video.add_overlays(overlays)

    # Instantiate and return the JupyterPlayer
    player = JupyterPlayer(video)
    return player

player = get_jupyter_player()
player.show()  # Display the player in Jupyter Notebook

In [ ]:
def get_player():
    """Returns a configured JupyterPlayer instance for interactive use."""
    video = Video(video_path)
    
    # Add named transformations
    video.add_transform(name="Grey scale", transform_func=lambda frame: cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    video.add_transform(name="Canny Edges", transform_func=lambda frame: cv2.Canny(frame, 100, 200))
    video.add_transform(name="Threshold", transform_func=lambda frame: cv2.threshold(frame, 127, 255, cv2.THRESH_BINARY)[1])
    video.add_transform(name="resize", transform_func=lambda frame: cv2.resize(frame, (960 // 2, 540 // 2)))

    video.add_online_overlay(
        name="Features", 
        overlay_func=lambda frame: np_to_overlay_items(
            cv2.goodFeaturesToTrack(frame, maxCorners=50, qualityLevel=0.01, minDistance=10, blockSize=3),
            overlay_item=Point,
        ),
    )

    # Load bounding boxes from the CSV file with a specific overlay name
    overlays = load_bounding_boxes_from_csv(annotations_path, overlay_name="CSV Boxes")
    video.add_overlays(overlays)

    # Instantiate and return the JupyterPlayer
    player = DesktopPlayer(video)
    return player

normal_player = get_player()
normal_player.show()  # Display the player in Jupyter Notebook

In [ ]:
import numpy as np
from video_streamer import Streamer

streamer = Streamer(video_path)
frames = []
for chunk in streamer.stream(start=0, end=10, chunk_size=5, overlap=0, unit="seconds"):
    print(f"Chunk: {chunk.start} to {chunk.end}, Frames: {len(chunk.frames)}")
    frames.extend(chunk.frames)

frames = np.array(frames)

In [ ]:
a=lambda frame: np_to_overlay_items(
            cv2.goodFeaturesToTrack(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), maxCorners=50, qualityLevel=0.01, minDistance=10, blockSize=3),
            overlay_item=Point
)
a(frames[0])